In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip /content/gdrive/MyDrive/archive.zip

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import callbacks 
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

In [ ]:
train_data = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_data = '/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid'

In [ ]:
# Define data generators with data augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_gen = train_datagen.flow_from_directory(
    train_data,
    target_size=(256, 256),
    batch_size=64,
    class_mode='sparse'
)
test_gen = test_datagen.flow_from_directory(
    valid_data,
    target_size=(256, 256),
    batch_size=64,
    class_mode='sparse'
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [ ]:
# Define model with regularization and transfer learning
base_model = keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(256, 256, 3)
)

for layer in base_model.layers[:-4]:
    layer.trainable = False

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5), #mengurangi overfitting
    keras.layers.Dense(38, activation='softmax')
])

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 1024)              1311744   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 38)                38950     
                                                                 
Total params: 3,608,678
Trainable params: 1,763,494
No

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/MyDrive/Model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

history = model.fit(train_gen, validation_data=test_gen, epochs=25, callbacks=my_callbacks)

Epoch 1/25


KeyboardInterrupt: ignored

In [ ]:
"""
Komentar Advisor : Bagian ini kalian yang tambahkan sendiri ?
Jawaban team : Iya, benar Kak kami menambahkan sendiri, apa salah ya Kak?
"""
plant_name = ['Apple', 'Apple', 'Apple', 'Apple', 'Blueberry', 'Cherry', 'Cherry', 'Corn', 'Corn', 'Corn', 'Corn', 
              'Grape', 'Grape', 'Grape', 'Grape', 'Orange', 'Peach', 'Peach', 'Pepper Bell', 'Pepper Bell', 'Potato', 'Potato', 'Potato',
              'Raspberry', 'Soybean', 'Squash', 'Strawberry', 'Strawberry', 'Tomato', 'Tomato', 'Tomato', 'Tomato', 'Tomato',
               'Tomato', 'Tomato', 'Tomato', 'Tomato', 'Tomato'
              ]

In [ ]:
"""
Komentar Advisor : Bagian ini kalian yang tambahkan sendiri ?
Jawaban team : Iya, benar Kak kami menambahkan sendiri, apa salah ya Kak?
"""
plant_disease = ['Scab', 'Black Rot', 'Cedar Apple Rust', 'Healthy', 'Healthy', 'Powdery Mildew', 'Healthy', 'Cercospora Leaf Spot Gray Leaf Spot', 'Comman Rust', 'Northern Leaf Blight', 
                 'Healthy', 'Black Rot', 'Esca', 'Leaf Blight', 'Healthy', 'Haunglongbing', 'Bacterial SPot',
                 'Healthy', 'Bacterial Spot', 'Healthy', 'Early Blight', 'Late Blight', 'Healthy', 'Healthy', 'Healthy', 'Powdery Mildew', 
                 'Leaf Scorch', 'Healthy', 'Bacterial Spot', 'Early Blight', 'Late Blight', 'Leaf Mold', 'Septoria Leaf', 
                 'Spider Mites', 'Target Spot', 'Tomato Yellow Leaf Curl Virus', 'Tomato Mosaic Virus', 'Healthy']

In [ ]:
print(plant_disease[4])

Powdery Mildew


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load trained model
model_path = '/content/gdrive/MyDrive/Model.18-0.08.h5'  # Ganti dengan path model yang ingin Anda gunakan
model = load_model(model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
model_path = '/content/gdrive/MyDrive/Model/model.tflite'
with open(model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Input gambar untuk prediksi
image_path = '/content/bercak-daun-septoria.jpg'  # Ganti dengan path gambar yang ingin Anda uji
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = image_array / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Melakukan prediksi dengan model
predictions = model.predict(image_array)
class_index = np.argmax(predictions[0])

print("Prediksi kelas:", class_index)

1/1 [==============================] - 2s 2s/step
Prediksi kelas: 32


In [ ]:
# Input gambar untuk prediksi
image_path = '/content/apple scab.jpeg'  # Ganti dengan path gambar yang ingin Anda uji
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = image_array / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Melakukan prediksi dengan model
predictions = model.predict(image_array)
class_index = np.argmax(predictions[0])

print("Prediksi kelas:", class_index)

1/1 [==============================] - 0s 25ms/step
Prediksi kelas: 26


In [ ]:
# Input gambar untuk prediksi
image_path = '/content/scab.JPG'  # Ganti dengan path gambar yang ingin Anda uji
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = image_array / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Melakukan prediksi dengan model
predictions = model.predict(image_array)
class_index = np.argmax(predictions[0])

print("Prediksi kelas:", class_index)

1/1 [==============================] - 0s 35ms/step
Prediksi kelas: 0
